# TitaniumCloud sample management API-s
This notebook contains code examples for the API-s that enable the user to manage their file samples.
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API classes:
- **FileUpload** (*TCA-0202-0203 - File Upload*)
- **FileDownload** (*TCA-0201 - File Download*)
- **ReanalyzeFile** (*TCA-0205 - Re-Analyze File*)
- **DeleteFile** (*TCA-0204 - Delete File*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API classes from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import FileUpload, FileDownload, ReanalyzeFile, DeleteFile

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**


In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")

### 3. File upload
The File upload API enables the user exactly what its name implies - to upload a new file sample to the TitaniumCloud.  

To upload a file, place it in the same directory as this notebook/Python script.  
Next, **replace the placeholder file name** in the `FILE_NAME` variable with your actual file name **including the file type extension**.

In [ ]:
FILE_NAME = "file_name_placeholder"

file_upload = FileUpload(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

upload_response = file_upload.upload_sample_from_path(file_path=FILE_NAME)

status_code = upload_response.status_code
print(status_code)

Since it only handles file uploads, this API does not return any specific response body text. To verify that our file was successfully submitted for upload we checked the request's status code. If the status code is `200` it means that our action was successful.


### 4. File download
File download enables us two actions:
- Checking the download status of a file
- Downloading the file

Here we will demonstrate how to download a file from the TitaniumCloud to your computer. All the steps from the code will be explained below.

In [ ]:
FILE_HASH = "file_hash_placeholder"

file_download = FileDownload(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

download_response = file_download.download_sample(hash_input=FILE_HASH)

with open("downloaded_file", "wb") as file_handle:
    file_handle.write(download_response.content)

For downloading a file, we need to know its hash string. To use this code snippet, you have to replace the file hash placeholder in the `FILE_HASH` variable with an actual hash.  
What the remaining code from the snippet does is the following:
- It downloads the specified file's binary content from the TitaniumCloud into the `download_response` variable.
- It opens up a new empty file called `downloaded_file` in this current directory.
- It writes the downloaded binary content into the empty `downloaded_file`.


### 5. Reanalyze file
Files in the TitaniumCloud are being periodically analyzed all the time. However, in case we want to manually run another analysis on our file, we can use the `ReanalyzeFile` class.

In [ ]:
reanalyze = ReanalyzeFile(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

reanalyze_response = reanalyze.reanalyze_samples(sample_hashes=FILE_HASH)

response_text = reanalyze_response.text
print(response_text)

For this we also used the same previously defined hash which represents our file in the TitaniumCloud. If the reanalyze request was successful the printed out response text will state the following:  
`Sample sent for rescanning`


### 6. Delete file
In case we no longer want a file to be present in the cloud and available for analysis or download, we can delete it.
To delete a file from the TitaniumCloud, we will use the `DeleteFile` class.

In [ ]:
delete_file = DeleteFile(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent="ReversingLabs SDK cookbook"
)

delete_response = delete_file.delete_samples(sample_hashes=FILE_HASH)

response_text = delete_response.text
print(response_text)

Again, we used the same file hash. If our file deletion request was successful, the response text will have a non-empty list called `deleted_hashes`. This list carries the hashes of files that have successfully been deleted with this request.